In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from modules import AthenaQueryExecutor, SqlRender
import pytz
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

from extra.config import awsconfig
from extra.utils import (extract, load, utils)
from extra.constants import (trx_columns, data_id_pago, name_columns_masivo)
from extra import transforms

warnings.filterwarnings('ignore')
# plt.style.use("../../0. Matriz tarjeta - Proyecto base/dataviz_template/corporate.mplstyle")
# pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# Obtener la fecha y hora actual
fecha_hora_actual = datetime.now()

# Configurar la zona horaria a 'America/Bogota'
zona_horaria_colombia = pytz.timezone('America/Bogota')
fecha_hora_colombia = fecha_hora_actual.astimezone(zona_horaria_colombia)

# fecha_result = fecha_hora_colombia - relativedelta(months=3)
# print(fecha_result.year)

string_filtro = ""

# creando el string_filtro con las fechas desde 3 meses atras.
for i in range(3):
    fecha_result = fecha_hora_colombia - relativedelta(months=i)
    string_filtro = string_filtro + "(year = '" + str(fecha_result.year) + "' AND month = '"+str(fecha_result.month).zfill(2)+"') OR "
string_filtro = string_filtro[:-4]
print(string_filtro)

(year = '2025' AND month = '01') OR (year = '2024' AND month = '12') OR (year = '2024' AND month = '11')


In [3]:
awsconfig.aws_config_profile('NEQUI-DataBusinessAnalyst-Role-044879804046')

Perfil AWS configurado: NEQUI-DataBusinessAnalyst-Role-044879804046


In [4]:
query_malla_prot_crisis = SqlRender.render_template(name_template="datos_tickets_all.sql",database="co_delfos_servicio_raw_pdn_rl",
                                                    table="co_zendesk_flatten_tickets", filtro_fecha=string_filtro)

print(query_malla_prot_crisis)

df_consulta = AthenaQueryExecutor.execute_query(query=query_malla_prot_crisis, database_athena="co_delfos_servicio_raw_pdn_rl", 
                                                workgroup_athena="delfos")


SELECT
        ticket_id AS "ID_del_ticket",
        fields_payload['numero_de_documento'] as Numero_de_documento,
        fields_payload['modalidad_de_fraude'] as Modalidad_de_fraude,
        fields_payload['tipo_de_incidente'] as Tipo_de_incidente,
        created_at as "Ticket_creado_Fecha",
        type AS "Tipo_de_ticket",
        CASE
            WHEN type = 'question' THEN 0
            ELSE CAST(fields_payload['Valor_del_fraude'] AS INTEGER)
        END AS "Valor_del_fraude"
FROM co_delfos_servicio_raw_pdn_rl.co_zendesk_flatten_tickets
WHERE true
        AND ((year = '2025' AND month = '01') OR (year = '2024' AND month = '12') OR (year = '2024' AND month = '11'))
        AND ticket_form_label = 'gestion_de_fraude'
        AND brand_id = 1088748
        AND type IN ('incident', 'question') --Tipo ticket

        AND ( (type IN ('incident')) AND (fields_payload['tipo_de_incidente'] in ('queja_smart','reclamo_smart', 'solicitud_smart')) )

        and ((fields_payload['fusionado']

In [5]:
cols = ['ID del ticket', 'Número de documento', 'Modalidad', 'Tipo de incidente', 
        'Ticket creado - Fecha', 'Tipo de ticket', 'Valor del fraude']

rename_columns = {  'ID_del_ticket':'ID del ticket',
                    'Numero_de_documento':'Número de documento',
                    'Modalidad_de_fraude':'Modalidad',
                    'Tipo_de_incidente':'Tipo de incidente',
                    'Ticket_creado_Fecha':'Ticket creado - Fecha',
                    'Tipo_de_ticket':'Tipo de ticket',
                    'Valor_del_fraude':'Valor del fraude'
                }

# rename_columns = {  'Modalidad de fraude':'Modalidad'}

df_consulta = df_consulta.rename(columns=rename_columns)

df = df_consulta.copy()


# Concateno todas las modalidades
df['Ticket creado - Fecha'] = pd.to_datetime(df['Ticket creado - Fecha'])
df['Anio-Mes'] = df['Ticket creado - Fecha'].dt.strftime('%Y-%m')

print(df.shape)

(51199, 8)


In [6]:
df.groupby('Modalidad')['Modalidad'].count()

Modalidad
compras_no_reconocidas_con_tarjeta_débito_modalidad     5716
estafa_modalidad                                         261
listas_restrictivas_bloqueos                             958
monitoreo_transaccional_bloqueos_alertas                2129
receptores_de_fraude_modalidad                          1867
suplantación_de_identidad_modalidad                     3053
vulneración_de_cuentas_modalidad                       37215
Name: Modalidad, dtype: int64

## Analisis de registros que se eliminarian

In [7]:
# lista=df.dropna(subset=['Número de documento', 'Tipo de incidente', 'Modalidad', 'Ticket creado - Fecha'])['ID del ticket'].tolist()
# df_result = df[~df['ID del ticket'].isin(lista)]
# df_result.groupby('Modalidad')['Modalidad'].count()

# Asi estaba en el codigo
# print(f"Tamaño inicial de la muestra: {df.shape[0]}")
# df = df.dropna(subset=['Número de documento', 'Tipo de incidente', 'Modalidad', 'Ticket creado - Fecha'])
# print(f"Tamaño de la muestra eliminando campos vacíos: {df.shape[0]}\n")

In [8]:
# Pruebas de condicional
print(f"Tamaño inicial de la muestra: {df.shape[0]}")
df['Valor del fraude'] = df['Valor del fraude'].astype('Float64')

percentil = 99
umbral = np.percentile(df['Valor del fraude'].dropna(), percentil)

print(f"Percentil {percentil} de valores de fraude : {round(umbral, 2)}")
print(f"Se pierden un {round(df[((df['Valor del fraude'] > umbral) & (df['Valor del fraude'].notna()) & (df['Valor del fraude'] != 0))].shape[0]/df.shape[0]*100, 2)}% de los datos")

df = df[~((df['Valor del fraude'] > umbral) & (df['Valor del fraude'].notna()) & (df['Valor del fraude'] != 0))]
print(f"Tamaño de la muestra limpia: {df.shape[0]}\n")

Tamaño inicial de la muestra: 51199


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
## Agrupación por cada evento (Número de documento, Modalidad de reclamo Gestión de Fraude y Ticket creado - Fecha )
df = df.groupby(['Número de documento', 'Tipo de incidente', 'Modalidad', 'Ticket creado - Fecha']).agg({'ID del ticket': 'count', 'Valor del fraude': 'sum'}).sort_values('ID del ticket').reset_index()
df['Anio-Mes'] = df['Ticket creado - Fecha'].dt.strftime('%Y-%m')
df['Número de documento'] = df['Número de documento'].astype(str)
print(f"Tamaño de la muestra agrupada: {df.shape[0]}\n")
df.to_parquet("./data/processed/data_eventos_fraude.parquet", engine='pyarrow')

df.to_excel("./data/processed/data_eventos_fraude_2.xlsx")
df
